In [6]:
import sys
sys.path.append('src/')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch
import ast
import matplotlib.pyplot as plt
import numpy as np
import evaluate
from datasets import Dataset

# panda view settings
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns

In [15]:
combined_training_bart = pd.read_csv('results/training/bart-large/combined_training_results.csv')
combined_training_debertav3 = pd.read_csv('results/training/debertav3/combined_deberta_trainng_results.csv')

## Test f1 scores

In [16]:
def set_predictions_multiple(df, max_correct_threshold=1):
    def is_bias(row, n_labels):
        labels = ast.literal_eval(row['labels'])
        if sum(label == 'entailment' for label in labels) >= n_labels:
            return 'bias'
        else:
            return 'neutral'

    label_correctness_list = []

    for _, row in df.iterrows():
        row_results = []

        for n in range(1, max_correct_threshold + 1):
            result = is_bias(row, n)
            row_results.append(result)

        label_correctness_list.append(row_results)

    # Add the list as the 'is_label_correct' column in the DataFrame
    df['predicted'] = pd.Series(label_correctness_list)

    return df


In [17]:
df_training_multiple_bart = set_predictions_multiple(combined_training_bart, 15)
df_training_multiple_debertav3 = set_predictions_multiple(combined_training_debertav3, 15)